# Prompting Best Practices and Common Use Cases

In [1]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI

load_dotenv(".env", override=True)

True

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
client = OpenAI()

In [4]:
def get_response(prompt, model="gpt-4.1-nano"):

    response = client.responses.create(
    model=model,
    input=prompt,
    temperature = 0)
    
    return response.output_text

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**


# Principle 1: Write clear and specific instructions

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [5]:
text = """
You should express what you want a model to do by 
providing instructions that are as clear and 
specific as you can possibly make them. 
This will guide the model towards the desired output, 
and reduce the chances of receiving irrelevant 
or incorrect responses. Don't confuse writing a 
clear prompt with writing a short prompt. 
In many cases, longer prompts provide more clarity 
and context for the model, which can lead to 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks 
into a single sentence.
```{text}```
"""
response = get_response(prompt)
print(response)

Clear and specific instructions in prompts help guide the model to produce accurate and relevant responses, with longer prompts often providing better clarity and context.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [6]:
prompt = """
Generate a list of three made-up book titles along 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_response(prompt)
print(response)

```json
[
  {
    "book_id": 1,
    "title": "The Echoes of Midnight",
    "author": "Lila Montgomery",
    "genre": "Mystery"
  },
  {
    "book_id": 2,
    "title": "Starlight and Shadows",
    "author": "Javier Torres",
    "genre": "Fantasy"
  },
  {
    "book_id": 3,
    "title": "Waves of Memory",
    "author": "Sophie Chen",
    "genre": "Historical Fiction"
  }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [7]:
text_1 = """
Making a cup of tea is easy! First, you need to get some 
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is
hot enough, just pour it over the tea bag. 
Let it sit for a bit so the tea can steep. After a 
few minutes, take out the tea bag. If you 
like, you can add some sugar or milk to taste. 
And that's it! You've got yourself a delicious 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, 
then simply write "No steps provided."

\"\"\"{text_1}\"\"\"
"""
response = get_response(prompt)

print(response)

Step 1 - Get some water boiling.  
Step 2 - Grab a cup and put a tea bag in it.  
Step 3 - Once the water is hot enough, pour it over the tea bag.  
Step 4 - Let the tea steep for a few minutes.  
Step 5 - Take out the tea bag.  
Step 6 - Add sugar or milk if desired.


In [8]:
text_2 = """
The sun is shining brightly today, and the birds are 
singing. It's a beautiful day to go for a 
walk in the park. The flowers are blooming, and the 
trees are swaying gently in the breeze. People 
are out and about, enjoying the lovely weather. 
Some are having picnics, while others are playing 
games or simply relaxing on the grass. It's a 
perfect day to spend time outdoors and appreciate the 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided."

\"\"\"{text_2}\"\"\"
"""
response = get_response(prompt)

print(response)

No steps provided.


#### Tactic 4: "Few-shot" prompting

In [9]:
prompt = """
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest 
valley flows from a modest spring; the
grandest symphony originates from a single note; 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_response(prompt)
print(response)

<grandparent>: The mighty oak stands tall through storms and winds; the strongest steel is forged in the hottest fires; the brightest stars shine through the darkest night.


# Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [10]:
text = """
In a charming village, siblings Jack and Jill set out on 
a quest to fetch water from a hilltop 
well. As they climbed, singing joyfully, misfortune 
struck—Jack tripped on a stone and tumbled 
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to 
comforting embraces. Despite the mishap, 
their adventurous spirits remained undimmed, and they 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple 
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following 
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_response(prompt_1)

print(response)

Jack and Jill, two siblings from a charming village, went on an adventure to fetch water but faced a mishap when Jack tripped and fell, yet they remained cheerful and continued exploring.

---

Jack et Jill, deux frères et sœurs d'un village charmant, sont partis à l'aventure pour aller chercher de l'eau mais ont rencontré un incident lorsque Jack a trébuché et est tombé, mais ils sont restés joyeux et ont continué à explorer.

---

Noms dans le résumé français : Jack, Jill

---

```json
{
  "french_summary": "Jack et Jill, deux frères et sœurs d'un village charmant, sont partis à l'aventure pour aller chercher de l'eau mais ont rencontré un incident lorsque Jack a trébuché et est tombé, mais ils sont restés joyeux et ont continué à explorer.",
  "num_names": 2
}
```


#### Ask for output in a specified format

In [11]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_response(prompt_2)
print(response)

Summary: Jack and Jill, two siblings from a village, experienced a fall while fetching water but remained adventurous and continued exploring despite their mishap.

Translation: Jack et Jill, deux frères et sœurs d'un village, ont eu une chute en allant chercher de l'eau mais sont restés aventureux et ont continué à explorer malgré leur mésaventure.

Names: Jack, Jill

Output JSON: 
```json
{
  "french_summary": "Jack et Jill, deux frères et sœurs d'un village, ont eu une chute en allant chercher de l'eau mais sont restés aventureux et ont continué à explorer malgré leur mésaventure.",
  "num_names": 2
}
```


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [12]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need 
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 / square 
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_response(prompt)
print(response)

Let's verify the student's solution step by step:

Given:
- Land cost: \$100 per sq ft
- Solar panels: \$250 per sq ft
- Maintenance: \$100,000 flat + \$10 per sq ft

Let \( x \) be the size of the installation in square feet.

Calculations:
1. Land cost: \( 100x \)
2. Solar panel cost: \( 250x \)
3. Maintenance cost: \( 100,000 + 10x \)

Total cost:
\[
100x + 250x + (100,000 + 10x) = (100x + 250x + 10x) + 100,000 = 360x + 100,000
\]

The student combined the costs as:
\[
100x + 250x + 100,000 + 100x = 450x + 100,000
\]

This is incorrect because the maintenance cost includes an additional \( 10x \), which the student did not include. Instead, the total should be:

\[
(100x + 250x + 10x) + 100,000 = 360x + 100,000
\]

**Conclusion:**
The student's total cost calculation is **incorrect**. The correct total cost for the first year as a function of \( x \) is:

\[
\boxed{360x + 100,000}
\]


In [14]:
prompt = f"""
Your task is to determine if the student's solution 
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution 
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help 
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost 
me a flat $100k per year, and an additional $10 / square 
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_response(prompt)
print(response)

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
```
Let's break down each component:

1. Land cost: $100 per square foot, so for x square feet: 100x
2. Solar panel cost: $250 per square foot, so for x square feet: 250x
3. Maintenance cost: a flat $100,000 plus $10 per square foot, so: 100,000 + 10x

Adding all these costs together:
Total cost = Land + Solar panels + Maintenance
= 100x + 250x + (10

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [15]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_response(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a modern, high-tech toothbrush designed to enhance oral hygiene. It features a sleek, ultra-slim design that makes it comfortable to hold and easy to maneuver. The "Smart" aspect typically indicates that it includes connectivity features, such as Bluetooth, allowing users to track their brushing habits via a companion app. This can help improve brushing techniques, ensure thorough cleaning, and promote better dental health.

Key features of the AeroGlide UltraSlim Smart Toothbrush may include:
- **Slim, ergonomic design** for comfortable handling.
- **Smart connectivity** to monitor brushing patterns and provide feedback.
- **Multiple brushing modes** for customized cleaning (e.g., sensitive, deep clean).
- **Timer and pacer functions** to ensure proper brushing duration.
- **Replaceable brush heads** for convenience and sustainability.
- **Rechargeable battery** with long-lasting power.

Boie is known for creating innovative, user-fr